In [4]:
import numpy
import scipy.special
import matplotlib.pyplot

%matplotlib inline

In [5]:
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningRate):
        # set number of nodes in input, hidden, and output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # learning rate
        self.lr = learningRate
        
        # activiation function (sigmoid)
        self.activationFunction = lambda x: scipy.special.expit(x)
        
        # init weights
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
    
    
    
    def train(self, inputsList, targetsList):
        # convert inputs and targets list into 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        targets = numpy.array(targetsList, ndmin=2).T
        
        # multiply weights with inputs
        hiddenInputs = numpy.dot(self.wih, inputs)
        # apply activation function
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # multiply weights with hidden outputs
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # apply activation function
        finalOutputs = self.activationFunction(finalInputs)
        
        
        # error is target - expected
        outputErrors = targets - finalOutputs
        
        # hidden layer error is the output errors, split by weights, recombined at hidden nodes
        hiddenErrors = numpy.dot(self.who.T, outputErrors)
        
        
        # update weights between hidden and output nodes
        self.who += self.lr * numpy.dot((outputErrors * finalOutputs * (1.0 - finalOutputs)), numpy.transpose(hiddenOutputs))
        
        # update weights between input and hidden nodes
        self.wih += self.lr * numpy.dot((hiddenErrors * hiddenOutputs * (1.0 - hiddenOutputs)), numpy.transpose(inputs))
    
    
    
    def query(self, inputsList):
        # convert inputs list into 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        
        # multiply weights with inputs
        hiddenInputs = numpy.dot(self.wih, inputs)
        # apply activation function
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # multiply weights with hidden outputs
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # apply activation function
        finalOutputs = self.activationFunction(finalInputs)
        
        return finalOutputs

In [6]:
# create a neural network
n = neuralNetwork(784, 100, 10, 0.3)

# load the training data
trainingDataFile = open('mnist_dataset/mnist_train_100.csv', 'r')
trainingDataList = trainingDataFile.readlines()
trainingDataFile.close()


# rescaling data and training nn
for d in trainingDataList:
    values = d.split(',')
    
    # scale to within 0.01 to 1.0
    inputs = (numpy.asfarray(values[1:]) / 255.0 * 0.99) + 0.01
    # create target of all 0.01 except expected which is 0.99
    targets = numpy.zeros(10) + 0.01
    targets[int(values[0])] = 0.99
    
    n.train(inputs, targets)

In [7]:
# load the testing data
testingDataFile = open('mnist_dataset/mnist_test_10.csv', 'r')
testingDataList = testingDataFile.readlines()
testingDataFile.close()

In [8]:
n.query((numpy.asfarray(values[1:]) / 255.0 * 0.99) + 0.01)

array([[ 0.02005888],
       [ 0.92828106],
       [ 0.03554777],
       [ 0.06751976],
       [ 0.01918867],
       [ 0.04922449],
       [ 0.0116457 ],
       [ 0.08850418],
       [ 0.07310606],
       [ 0.01041111]])